In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras

In [ ]:
test_batch_size = 8
image_size = 512

In [ ]:
PATH =  '../input/cassava-leaf-disease-classification/'
sub_df = pd.read_csv(PATH + 'sample_submission.csv')

In [ ]:
sub_df["path"] = PATH + 'test_images/' + sub_df["image_id"]

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((sub_df.path.values, sub_df.label.values))

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
from albumentations import (Compose, RandomCrop,Transpose, HorizontalFlip, 
                            VerticalFlip, ShiftScaleRotate, HueSaturationValue,   
                            RandomBrightness,RandomContrast, CenterCrop, ToFloat)

test_transforms = Compose([
            CenterCrop(image_size, image_size),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            ToFloat(max_value=255)
        ])

In [ ]:
def test_aug(image):
    data = {"image":image}
    aug_data = test_transforms(**data)
    aug_img = aug_data["image"]
    aug_img = tf.cast(aug_img, tf.float32)
    #aug_img = tf.image.resize(aug_img, size=[512, 512])    
    return aug_img

In [ ]:
def process_data(image_path, label, func_aug):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    aug_img = tf.numpy_function(func=func_aug, inp=[image], Tout=tf.float32)
    return aug_img, label

In [ ]:
from functools import partial
test_ds_alb = test_ds.map(partial(process_data, func_aug=test_aug), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [ ]:
def set_shapes(img, label, img_shape=(512, 512,3)):
    img.set_shape(img_shape)
    label.set_shape([])
    return img, label

In [ ]:
test_ds_alb = test_ds_alb.map(set_shapes, num_parallel_calls=AUTOTUNE).batch(8)

In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

In [ ]:
import efficientnet.tfkeras as efn 
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D


efficientnet = efn.EfficientNetB4(weights=None, include_top=False,
                                  drop_connect_rate=0.3, input_shape=(image_size, image_size, 3))

inputs = Input(shape=(image_size, image_size, 3))
efficientnet = efficientnet(inputs)
pooling = GlobalAveragePooling2D()(efficientnet)
dropout = Dropout(0.3)(pooling)
outputs = Dense(5, activation="softmax")(dropout)
model = Model(inputs=inputs, outputs=outputs)

In [ ]:
preds = []
for fold in [2,3]:
    model.load_weights('../input/effnetb45128weights/EffNetB4_512_8_weights_fold_{}.h5'.format(fold+1))
    preds.append(model.predict(test_ds_alb, workers=4, verbose=1))

In [ ]:
preds = []

for fold in [2,3]:
    model.load_weights('../input/effnetb45128weights/EffNetB4_512_8_weights_fold_{}.h5'.format(fold+1))
    fold_preds = []
    for i in range(5):
        fold_preds.append(model.predict(test_ds_alb, workers=4, verbose=1))
    preds.append(np.mean(fold_preds, axis=0))

In [ ]:
preds_avg = np.mean(preds, axis=0) 

In [ ]:
y_pred = np.argmax(preds_avg, axis=-1)

In [ ]:
sub_df['label'] = y_pred

In [ ]:
sub_df.drop(['path'], axis=1, inplace=True)

In [ ]:
sub_df.to_csv('submission.csv', index=False)